In [1]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import pandas as pd
import numpy as np
import json

In [90]:
def crawler_boletins(dia, mes, ano):

    # Firefox WebDrive
    options = Options()
    options.headless = False    #Not open the broser 
    driver = webdriver.Firefox(options=options, executable_path = r'C:\Users\upe2\bin\geckodriver.exe')

    # Open CPTEC wepage
    driver.get("http://tempo.cptec.inpe.br/boletimtecnico/pt")

    #Get the CSS frame for select reports ans switch context
    frame = driver.find_element_by_css_selector('div.cont_dir iframe')
    driver.switch_to.frame(frame)

    #Select date and press enter
    elem = driver.find_element_by_id("j_id_jsp_31316922_1:dia")
    if dia != 1:
        for n in range(dia-1):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:mes")
    if mes != 1:
        for n in range(mes-1):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:ano")
    if ano != 2021:
        for n in range(2021 - ano):
            elem.send_keys(Keys.ARROW_DOWN)

    elem = driver.find_element_by_id("j_id_jsp_31316922_1:submit")
    elem.send_keys(Keys.RETURN)

    #Back the context to former frame
    driver.switch_to.parent_frame()

    #Get the CSS frame for select reports ans switch context
    frame = driver.find_element_by_css_selector('div.cont_meio iframe')
    driver.switch_to.frame(frame)
    
    try:
        
        #Convert dia, mes, ano in string
        # Dia
        if len(str(dia)) == 2:
            str_dia = str(dia)
        else:
            str_dia = "0" + str(dia)
        # Mes
        if len(str(mes)) == 2:
            str_mes = str(mes)
        else:
            str_mes = "0" + str(mes)
            
        #Ano
        str_ano = str(ano)
            
        
        element = WebDriverWait(driver, 3).until(
            EC.text_to_be_present_in_element((By.TAG_NAME, "em"), 
                                             "Análise Sinótica: " + str_dia + "/" + str_mes + "/" + str_ano + "-00Z")
        )
        
        #get the report texts
        elem = driver.find_elements_by_class_name("texto")
        texts = []
        for cell in elem:
            texts.append(cell.text)

        elem = driver.find_elements_by_class_name("sub")
        col = []
        for cell in elem:
            col.append(cell.text)

        # Close CPTEC wepage
        driver.close()

        return(col, texts,)
    
    except:
        driver.close()
        return ([[],[]])

In [91]:
dia = 10
mes = 3
ano = 2011

texto = crawler_boletins(dia, mes, ano)
texto

Análise Sinótica: 10/03/2011-00Z


(['Análise Sinótica: 10/03/2011-00Z',
  'Nível 250 hPA',
  'Nível 500 hPA',
  'Nível 850 hPA',
  'Superfície',
  'Previsão'],
 ['Imagem de Satélite Goes 16',
  'Na análise da carta sinótica de altitude da 00Z (250 hPa) de hoje 10/03, observa-se um crista sobre o país, entre os paralelos 10 e 20S, compreendendo boa parte do Centro-Oeste, Sudeste e sul do Nordeste. Este sistema desprende-se de um núcleo que posiciona-se sobre o Pacífico. Entre o MS, SP e Região Sul do Brasil o fluxo é de oeste, e mais ao sul, sobre o Uruguai, observa-se um significativo cavado, que foi um dos responsáveis por toda instabilidade observada entre a tarde de quarta-feira e o início do dia de hoje sobre o sul do estado gaúcho, como foi o caso de Rio Grande, onde o acumulado foi de 132mm. Os Jatos Polar Norte (JPN) e Polar Sul (JPS) contornam um cavado a sul de 50S, atuando sobre o extremo sul do continente e dando suporte a uma família de frente em superfície.\nVisualizar imagem de 250 hPA',
  'Na análise da 